## Fine-tune 
### procedure
1. convert the `train.json` into the format described below. In this case, we put back the correct answer into the question, then split this question with correct answer into two sentence, first sentence as the first line, second sentence put in the second line. please refer to the [lm_finetuning](https://github.com/huggingface/pytorch-pretrained-BERT/tree/master/examples/lm_finetuning)
2. Simple fine-tune 
```
python examples/lm_finetuning/simple_lm_finetuning.py 
    --train_batch_size 2 
    --train_corpus 
    ../corpus.txt 
    --bert_model 
    bert-base-uncased 
    --output_dir finetuned_lm 
    --do_train 
```
3. all the code based on library: https://github.com/huggingface/pytorch-pretrained-BERT

In [8]:
import json
import pprint
import collections
import re
from os.path import abspath, dirname, join


BASE_PATH = "/Users/shenweihai/Desktop/Projects/Ubuntu_exx/tutorial/bert-proj"
split_words = lambda s: s.split() 
strip_space = lambda s: re.sub("\s+", " ", re.sub("\s+\?", "?", re.sub("\s+\.", ".", re.sub("\s+,", ".", s)))).lstrip().rstrip().replace("..", ".")

def load_examples(file_name: str):
    ans = []
    err = collections.defaultdict(int)
    with open(file_name, 'r') as handler:
        for line in handler:
            example = json.loads(line)
            question = example['question']
            example = json.loads(line)
            question = example['question']
            
            if "(A)" not in question or "(B)" not in question:
                err['A_B'] += 1
                continue 
                
            idx = question.index("(A)")
            Q, second = question[0:idx], question[idx:]
                
            ans_idx = second.index("(B)")
            ans_first, ans_second = second[0:ans_idx].replace("(A)", ""), second[ans_idx:].replace("(B)", "")
            ans_first = re.sub("or\s*$", "", ans_first)
            ans_second = re.sub("or\s*$", "", ans_second)
            
            A = ans_first if example.get("answer_index", 0) == 0 else ans_second
            
            if "_____" in Q:
                Q = Q.replace("_____", A)
            else:
                Q = "{0} {1}.".format(Q, A)
            
            # split it into two sentence
            Q = re.sub(" +", " ", Q)
            
            Q_items = re.split("(\.|\?)", Q)
            Q_items_process = []
            for i in range(len(Q_items)):
                if Q_items[i] == '.' or Q_items[i] == '?':
                    Q_items_process[-1] = "{0}{1}".format(Q_items_process[-1], Q_items[i])
                else:
                    if Q_items[i]:
                        Q_items_process.append(Q_items[i])
    
            Q_items = [i for i in Q_items_process if i]
 
            Q_1, Q_2 = [], []
            threshold, tmp_w = 5, 0
            for idx, s in enumerate(Q_items[::-1]):
                if tmp_w <= threshold:
                    Q_2.append(s)
                else:
                    Q_1.append(s)
                tmp_w += len(split_words(s))
                            
            Q_1 = " ".join(Q_1[::-1]).rstrip()
            Q_2 = " ".join(Q_2[::-1]).rstrip()
            if not Q_1:
                Qs = re.split("(it\s+\w+|due to|so|because|and\s+\w+|it became|and it|this means|since|\?\s+|\.\s+|,\s+|but|it appeared)", Q_2)
                Q_1, Q_2 = " ".join(Qs[0:-2]), " ".join(Qs[-2:])
                Q_2 = re.sub(",\s+|\.\s+|\?\s+", "", Q_2)
            
            if not Q_2:
                err["Q_2"] += 1
            
            if not Q_1:
                err["Q_1"] += 1
            
            item = {
                "Q": question,
                "id": example['id'],
                "Q_1": strip_space(Q_1),
                "Q_2": strip_space(Q_2),
            }
            
            ans.append(item)
    print(err)
    return ans

ans = load_examples(join(BASE_PATH, "quarel-data", "quarel-v1-train.json"))

w = open("corpus.txt", "w")
for a in ans:
    if not a["Q_1"]: 
        continue

    w.write("{0}\n".format(a['Q_1']))
    w.write("{0}\n\n".format(a['Q_2']))
w.close()

pprint.pprint(ans[0:20])

defaultdict(<class 'int'>, {'Q_1': 14})
[{'Q': 'Mike was snowboarding on the snow and hit a piece of ice. He went much '
       'faster on the ice because _____ is smoother. (A) snow (B) ice',
  'Q_1': 'Mike was snowboarding on the snow and hit a piece of ice.',
  'Q_2': 'He went much faster on the ice because ice is smoother.',
  'id': 'QuaRel_V1_Fr_0223'},
 {'Q': 'John and Rita are going for a run.  Rita gets tired and takes a break '
       'on the park bench.  After twenty minutes in the park, who has run '
       'farther? (A) John (B) Rita',
  'Q_1': 'John and Rita are going for a run. Rita gets tired and takes a break '
         'on the park bench.',
  'Q_2': 'After twenty minutes in the park, who has run farther? John.',
  'id': 'QuaRel_V1_B5_1403'},
 {'Q': "The propeller on Kate's boat moved slower in the ocean compared to the "
       'river. This means the propeller heated up less in the (A) ocean (B) '
       'river',
  'Q_1': "The propeller on Kate's boat moved slower in t